In [1]:
import json

def load_batch(K=50, data_path='maps.json'):
    data = json.load(open(data_path))
    if K > len(data):
        raise IndexError("K should be no greater than size of dataset. size=" + str(len(data)))
    filenames = list(map(lambda x: x[0], data[:K]))
    objects = list(map(lambda x: x[1], data[:K]))
    feats = list(map(lambda x: x[2], data[:K]))
    return filenames, objects, feats

In [2]:
raw_names, raw_Y, raw_X = load_batch(2100)

In [5]:
with open('tmp.json', 'w') as f:
    json.dump((raw_names, raw_Y, raw_X), f)

In [1]:
import json
with open('tmp.json') as f:
    raw_names, raw_Y, raw_X = json.load(f)

In [3]:
X = raw_X

music_len = max(len(X[i]) for i in range(len(X)))
X_len = 65
Y_len = 4
num_hidden = 200

Y = [[[0, 0, 0, 0] for j in range(music_len)] for i in range(len(X))]
PX = [[0.5] * music_len for i in range(len(X))]
PY = [[0.5] * music_len for i in range(len(X))]
for i in range(len(X)):
    for j in range(music_len):
        if j < len(raw_Y[i]):
            Y[i][j][int(raw_Y[i][j][2])] = 1
            PX[i][j] = min(1, max(0, raw_Y[i][j][0]))
            PY[i][j] = min(1, max(0, raw_Y[i][j][1]))
        else:
            X[i].append([0]*X_len)

In [4]:
cnt = [0, 0, 0, 0]
for i in range(len(Y)):
    for j in range(len(Y[0])):
        for k in range(len(Y[0][0])):
            if Y[i][j][k] == 1:
                cnt[k] += 1
print cnt
print cnt[0] / 1.0 / sum(cnt), cnt[1] / 1.0 / sum(cnt), cnt[2] / 1.0 / sum(cnt), cnt[3] / 1.0 / sum(cnt)

[1372952, 415906, 1397773, 73114]
0.421183865609 0.12758850769 0.428798264895 0.0224293618059


In [5]:
num_train = 2000
t_names, t_X, t_Y, t_PX, t_PY = raw_names[num_train:], X[num_train:], Y[num_train:], PX[num_train:], PY[num_train:]
names, X, Y, PX, PY = raw_names[:num_train], X[:num_train], Y[:num_train], PX[:num_train], PY[:num_train]

In [6]:
import tensorflow as tf
data = tf.placeholder(tf.float32, [None, music_len, X_len])
target = tf.placeholder(tf.float32, [None, music_len, Y_len])
target_PX = tf.placeholder(tf.float32, [None, music_len])
target_PY = tf.placeholder(tf.float32, [None, music_len])

cell = tf.nn.rnn_cell.LSTMCell(num_hidden, state_is_tuple=True)
output, state = tf.nn.bidirectional_dynamic_rnn(cell, cell, data, sequence_length=[music_len]*100, dtype=tf.float32)
# output: [batch, length, feature]

In [7]:
weight = tf.Variable(tf.truncated_normal([2 * num_hidden, Y_len]))
bias = tf.Variable(tf.constant(0.0, shape=[Y_len]))

weight_PX = tf.Variable(tf.truncated_normal([2 * num_hidden, 1]))
bias_PX = tf.Variable(tf.constant(0.5, shape=[1]))

weight_PY = tf.Variable(tf.truncated_normal([2 * num_hidden, 1]))
bias_PY = tf.Variable(tf.constant(0.5, shape=[1]))

In [8]:
output2 = tf.concat(2, output)
val = tf.reshape(output2, [-1, num_hidden * 2])
tar = tf.reshape(target, [-1, Y_len])
tar_PX = tf.reshape(target_PX, [-1])
tar_PY = tf.reshape(target_PY, [-1])

preds = tf.matmul(val, weight) + bias
preds_PX = tf.sigmoid(tf.reshape(tf.matmul(val, weight_PX) + bias_PX, [-1]))
preds_PY = tf.sigmoid(tf.reshape(tf.matmul(val, weight_PY) + bias_PY, [-1]))

In [9]:
cross_ens = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(preds, tar))
l2_PX = tf.reduce_mean((preds_PX - tar_PX) ** 2)
l2_PY = tf.reduce_mean((preds_PY - tar_PY) ** 2)

losses = cross_ens + l2_PX + l2_PY

In [10]:
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(losses)

In [11]:
mistakes = tf.not_equal(tf.argmax(tar, 1), tf.argmax(preds, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

In [15]:
res, res_PX, res_PY = sess.run((preds, preds_PX, preds_PY), {data:t_X})

In [9]:
print music_len, #sess.run(data, dfm)

494


In [16]:
with open('filenames.json', 'w') as f:
    json.dump(t_names, f)
res.dump('labels')
res_PX.dump('PX')
res_PY.dump('PY')

In [12]:
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

In [13]:
batch_size = len(t_X)
no_of_batches = int(len(X) / batch_size)
dfm = {data:t_X, target:t_Y, target_PX:t_PX, target_PY:t_PY}
print len(X), no_of_batches

2000 20


In [ ]:
for i in range(30):
    ptr = 0
    for j in range(no_of_batches):
        inp, out, out_PX, out_PY = X[ptr:ptr+batch_size], Y[ptr:ptr+batch_size], PX[ptr:ptr+batch_size], PY[ptr:ptr+batch_size]
        ptr += batch_size
        ass = {data: inp, target: out, target_PX: out_PX, target_PY: out_PY}
        sess.run(minimize, ass)
#         incor = sess.run(error, ass)
#         loss = sess.run(losses, ass)
#         cross = sess.run(cross_ens, ass)
#         lx = sess.run(l2_PX, ass)
#         ly = sess.run(l2_PY, ass)
#         print('Epoch {:2d} loss {:3.6f}'.format(i + 1, loss))
#         print('         error {:3.6f}% cross {:3.6f} lx {:3.6f} ly {:3.6f}'.format(incor*100, cross, lx, ly))
#         print i, j
    incor, loss, cross, lx, ly = sess.run((error, losses, cross_ens, l2_PX, l2_PY), dfm)
    print('Epoch {:2d} loss {:3.6f}'.format(i + 1, loss))
    print('         error {:3.6f}% cross {:3.6f} lx {:3.6f} ly {:3.6f}'.format(incor*100, cross, lx, ly))

In [21]:
sess.close()